In [274]:
import os
import csv
import pandas as pd
from difflib import get_close_matches

import re

In [289]:
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main'

In [276]:
csv_leagues = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/transfermarkt_league_data_checking.csv')

In [290]:
csv_leagues['League'].value_counts()

League
Serie A                      9425
EPL                          8360
Turkish League               7883
Ligue 1                      7683
La Liga                      7404
Portuguese First Division    6983
Bundesliga                   6746
Eredivisie                   6713
Name: count, dtype: int64

In [277]:
def remove_leading_numeric(input_string):
    # Iterate through the string until a non-numeric character is encountered
    for i, char in enumerate(input_string):
        if not char.isdigit():
            # Remove leading numeric characters and spaces
            return input_string[i:].lstrip()
    
    # If the entire string is numeric, return an empty string
    return ""

# Example usage:
input_str = "1 I Casillas"
result = remove_leading_numeric(input_str)
print(result)

I Casillas


In [278]:
def remove_trailing_double_quote(input_string):
    # Check if the string ends with a double quote
    if input_string.endswith("\""):
        # Remove the trailing double quote
        return input_string[:-1]
    else:
        # Return the original string if no trailing double quote is found
        return input_string

In [279]:
def find_closest_string(input_string, string_list):
    # Use get_close_matches to find the closest matching string
    closest_match = get_close_matches(input_string, string_list, n=1, cutoff=0.8)

    # Check if a close match is found
    if closest_match:
        return closest_match[0]
    else:
        # If no close matches, extract the last word from the input_string
        last_word = input_string.split()[-1]

        # Return strings from the list if the last word is in those strings
        matching_strings = [s for s in string_list if last_word in s]

        if matching_strings:
            if(len(matching_strings) == 1):
                return matching_strings[0]
            else:
                
                return matching_strings
        else:
            return "No close match found."

In [280]:
def remove_dot_space_prefix(input_string):
    # Check if the string starts with ". "
    if input_string.startswith(". "):
        # Remove the ". " prefix
        return input_string[2:]
    else:
        # Return the original string if it doesn't start with ". "
        return input_string


In [281]:
from transliterate import translit

def cyrillic_to_latin(input_string):
    try:
        # Use the "translit" function to convert Cyrillic to Latin
        latin_string = translit(input_string, 'ru', reversed=True)
        return latin_string
    except Exception as e:
        # Handle exceptions, e.g., if the input is not valid Cyrillic
        print(f"Error: {e}")
        return input_string

In [282]:
def remove_last_two_chars(input_string):
    # Check if the string ends with a single character and then a period
    if len(input_string) >= 2 and input_string[-2].isalpha() and input_string[-1] == '.':
        # Remove the last two characters
        if(input_string[-3] == ' '):
            return input_string[:-3]
        else:
            return input_string[:-2]
    else:
        # Return the original string if it doesn't match the pattern
        return input_string

In [283]:
def is_cyrillic(input_string):
    # Check if the string contains non-ASCII characters
    return not input_string.isascii()

In [284]:
def process_string_newest_ii(input_string):
    cleaned_string = re.sub(r'^\d{1,2}[. ]', '', input_string)
    tokens = cleaned_string.split()
    final_string = ""

    if len(tokens) >= 1 and not re.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ]{2,}$', tokens[0]):
        initial_match = re.match(r'^([A-Za-zÀ-ÖØ-öø-ÿ]+\.)+$|[A-Za-zÀ-ÖØ-öø-ÿ]\.$|[A-Za-zÀ-ÖØ-öø-ÿ]$', tokens[0])
        if initial_match:
            final_string += initial_match.group()

    main_phrase = " ".join(word for word in tokens if len("".join(char for char in word if char.isalpha())) >= 2)
    if main_phrase:
        final_string += " " + main_phrase

    if len(tokens) >= 2 and not re.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ]{2,}$', tokens[1]):
        end_initial_match = re.match(r'^([A-Za-zÀ-ÖØ-öø-ÿ]+\.)+$|[A-Za-zÀ-ÖØ-öø-ÿ]\.$|[A-Za-zÀ-ÖØ-öø-ÿ]$', tokens[1])
        if end_initial_match:
            final_string = "".join(char for char in end_initial_match.group() if char.isalpha()) + " " + final_string

    # Check if the final phrase ends in a period
    if final_string.endswith("."):
        # Extract the last word, remove the period, and move it to the start of final_string
        last_word = final_string.split()[-1].rstrip('.')
        final_string = last_word + " " + final_string

        # Remove the last word from the end of the string
        final_string = ' '.join(final_string.split()[:-1])

        final_string = final_string.strip()

    # Separate the final string by " " and remove non-alphabet characters for each token
    final_tokens = [re.sub(r'[^A-Za-zÀ-ÖØ-öø-ÿ]', '', token) for token in final_string.split()]

    # If the first two tokens are the same, remove one token
    if len(final_tokens) >= 2 and final_tokens[0] == final_tokens[1]:
        final_tokens.pop(0)

    return " ".join(final_tokens)

In [285]:
def filter_candidates(NAMESTRING, LISTCANDIDATES):
    # Get the first token of the NAMESTRING
    first_token = re.split(r'\s', NAMESTRING)[0]

    # Create a regex pattern for matching candidates that start with the first token
    pattern = re.compile(fr'^{re.escape(first_token)}', re.IGNORECASE)

    # Filter candidates based on the pattern
    filtered_candidates = list(filter(lambda x: re.match(pattern, x), LISTCANDIDATES))

    return filtered_candidates

# Example usage:
name_string = "I Gueye"
candidates = ['Magaye Gueye',
  'Idrissa Gueye',
  'Moussa Gueye', 
  'Babacar Gueye',
  'Makhtar Gueye',
  'Pape Gueye',
  'Lamine Gueye']

filtered_candidates = filter_candidates(name_string, candidates)
#print(filtered_candidates)

In [287]:
for i in range(0, len(csv_leagues)):
    this_nationality = csv_leagues.at[i, 'Nationality']
    if(pd.isna(csv_leagues.at[i, 'Nationality'])):
        csv_leagues.at[i, 'Nationality'] = 'None'
    else:
        csv_leagues.at[i, 'Nationality'] = remove_trailing_double_quote(this_nationality)

In [294]:
example_problem = "10 Mane Sa."
natl_test = 'Senegal'
dataset_nationality = csv_leagues[csv_leagues['Nationality'] == f"{natl_test}"]['Name'].unique()

if(is_cyrillic(example_problem)):
    example_problem = cyrillic_to_latin(example_problem)


search_name = process_string_newest_ii(example_problem) 

result = find_closest_string(search_name, dataset_nationality)
if(type(result) == list):
    result = filter_candidates(search_name, result)[0]
search_name, result

('Sa Mane', 'No close match found.')

## OLD DEBUG

In [ ]:
input_str = '8 M Valbuena'
natl_test = 'France'

input_str = remove_leading_numeric(input_str)

dataset_nationality = csv_leagues[csv_leagues['Nationality'] == f"{natl_test}"]['Name'].unique()

result = find_closest_string(input_str, dataset_nationality)
result
# input_str_2 = "22. Bennacer I."
# natl_test = 'Algeria'

# input_str_2 = remove_leading_numeric(input_str_2)
# input_str_2 = remove_dot_space_prefix(input_str_2)
# input_str_2
# dataset_nationality = csv_leagues[csv_leagues['Nationality'] == f"{natl_test}"]['Name'].unique()

# result = find_closest_string(input_str_2, dataset_nationality)
# result    

In [153]:
string_test = '10. Ayew A.'
natl_test = 'Ghana'


string_test = remove_leading_numeric(string_test)
string_test = remove_dot_space_prefix(string_test)

string_test = remove_last_two_chars(string_test)

if(is_cyrillic(string_test)):
    string_test = cyrillic_to_latin(string_test)

dataset_nationality = csv_leagues[csv_leagues['Nationality'] == f"{natl_test}"]['Name'].unique()

result = find_closest_string(string_test, dataset_nationality)
result

['André Ayew', 'Jordan Ayew']